In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write

from g2s_with_question import pyopenjtalk_g2p_prosody_with_questioin



def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

## MB-iSTFT-VITS

In [11]:
# 使用したjsonファイルを指定する
config_path = "path/to/config.json"
hps = utils.get_hparams_from_file(config_path)

In [12]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

G_model_path = "path/to/G_xxxxxx.pth"
_ = utils.load_checkpoint(G_model_path, net_g, None)

Mutli-band iSTFT VITS


In [31]:
stn_phn = pyopenjtalk_g2p_prosody_with_questioin("特許許可する東京特許許可局。")
print(stn_phn)
stn_tst = get_text(stn_phn, hps)

with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

['^', 't', 'o', '[', 'cl', 'ky', 'o', ']', 'ky', 'o', 'k', 'a', '#', 's', 'u', '[', 'r', 'u', '#', 't', 'o', '[', 'o', 'ky', 'o', 'o', 't', 'o', 'cl', 'ky', 'o', 'ky', 'o', 'k', 'a', ']', 'ky', 'o', 'k', 'u', '$']
